# Library Imports

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import io
# from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score,\
                            precision_score, recall_score, accuracy_score,\
                            average_precision_score, precision_recall_curve
from imblearn.over_sampling import RandomOverSampler
from src.helper_functions import load_data, get_model_perfs, init_model_perfs,\
                                 save_model, save_model_perfs, check_is_best,\
                                 read_model, evaluate_model_predictions,\
                                 update_model_perfs, check_and_save,\
                                 adjusted_classes
from src.metrics import recall_m, f1

Using TensorFlow backend.
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/sean/anaconda3/envs

In [2]:
import keras
from keras.callbacks import Callback
from keras.layers import Dense, Activation, Dropout
keras.__version__

'2.2.5'

In [3]:
%load_ext autoreload
%autoreload 2

# Load Data

In [4]:
x_tr, y_tr, x_te, y_te = load_data()

# Build Neural Network

Following the desciption in section 2.2.4 of the [DeepTox article](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full), I tried to use intermediate values in [Table 2](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full#T2) to build the neural network:

Following [this question/answer](https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model) and [this question/answer](https://stackoverflow.com/questions/54065733/how-to-employ-the-scikit-learn-evaluation-metrics-functions-with-keras-in-python) to implement usage of recall in model training:

In [5]:
drop_out=0.5    # DeepTox range: 0.5, 0.2, 0
L2_reg = 0.0001 # Default = 0.01
layers = 3      # DeepTox range: 1, 2, 3, 4
act = 'sigmoid' # Consider sigmoid and tanh
neurons = 1024  # DeepTox range: 1024, 2048, 4096, 8192, 16384
# Info on decay: https://datascience.stackexchange.com/questions/26112/decay-parameter-in-keras-optimizers
decay = 0       # DeepTox range: 10^-4, 10^-5, 10^-6
learn_rate = 0.1  #Research appropriate range
DNN = keras.Sequential([
    keras.layers.InputLayer(input_shape=x_tr.shape[1:],name='Input_Layer')
])
for i in range(1,layers+1):
    DNN.add(Dense(units=neurons, activation=act,\
                  name='h'+str(i)+'_'+act+'_activation',\
                  kernel_regularizer=keras.regularizers.l2(L2_reg)))
    DNN.add(Dropout(rate=drop_out,name='Dropout'+str(i)))
DNN.add(Dense(units=1, activation='sigmoid'))
keras.optimizers.Adam(lr=learn_rate, beta_1=0.9,\
                      beta_2=0.999, decay=decay, amsgrad=False)
DNN.compile(optimizer='adam', loss='binary_crossentropy',\
            metrics=['accuracy',recall_m,f1])
DNN.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
h1_sigmoid_activation (Dense (None, 1024)              1684480   
_________________________________________________________________
Dropout1 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h2_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout2 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h3_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________

# Train DNN's

In [6]:
for target in y_tr.columns:
    # Determine rows with available data
    rows_tr = np.isfinite(y_tr[target]).values
    rows_te = np.isfinite(y_te[target]).values
    x,y = x_tr[rows_tr], y_tr[target][rows_tr]
    
    # Address Class Imbalance
    x_train, x_val, y_train, y_val = train_test_split(x, y, stratify=y, \
                                                      test_size=0.2, random_state=42)
    ros = RandomOverSampler(random_state=0)
    x_resampled, y_resampled = ros.fit_sample(x_train,y_train)
    
    # Train the DNN
    DNN.fit(
        x_resampled, y_resampled, batch_size=512, epochs=100,\
        validation_data=(x_val,y_val), verbose=1,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=16,verbose=1,\
                                          restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)
        ])
    
    # Get predictions, calculate model performance and save info
    p_te = DNN.predict(x_te[rows_te])[:,0]
    y_testing=y_te[target][rows_te]
    auc_te = roc_auc_score(y_testing, p_te)
    print("%15s: %3.5f" % (target, auc_te))

    y_hat_testing=DNN.predict_classes(x_te[rows_te])
    average_precision=average_precision_score(y_testing,p_te)
    mv=evaluate_model_predictions(target,'DNN',0.5,y_testing,y_hat_testing,\
                                  auc_te,average_precision)
    check_and_save(target,mv,DNN,True)
    
    # Find max F1 varying probability threshold, calculate modified performance, save
    precision, recall, thresholds = precision_recall_curve(y_testing,p_te)
    # f1 = 2*precision*recall/(precision+recall)  # Sometimes precision=recall=0!
    p,r,t = zip(*[i for i in zip(precision,recall,np.append(thresholds,1)) if i[0:2]!=(0,0)])
    p,r,t = np.array(p),np.array(r),np.array(t)
    f1 = 2*p*r/(p+r)
    m_idx = np.argmax(f1)
    m_thresh = thresholds[m_idx]
    y_hat_testing_adj=adjusted_classes(p_te,m_thresh)
    mv=evaluate_model_predictions(target,'DNN_modT',m_thresh,y_testing,y_hat_testing_adj,\
                                  auc_te,average_precision)
    check_and_save(target,mv,DNN,True)
    display(get_model_perfs(target))

Instructions for updating:
Use tf.cast instead.
Train on 11934 samples, validate on 1689 samples
Epoch 1/100
11934/11934 [==============================] - 3s 250us/step - loss: 1.1862 - acc: 0.5189 - recall_m: 0.5395 - f1: 0.4519 - val_loss: 0.7337 - val_acc: 0.8840 - val_recall_m: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 2/100
11934/11934 [==============================] - 2s 184us/step - loss: 0.8861 - acc: 0.5835 - recall_m: 0.5613 - f1: 0.5613 - val_loss: 0.9617 - val_acc: 0.4340 - val_recall_m: 0.9586 - val_f1: 0.2807
Epoch 3/100
11934/11934 [==============================] - 2s 183us/step - loss: 0.7973 - acc: 0.6548 - recall_m: 0.6999 - f1: 0.6685 - val_loss: 0.7909 - val_acc: 0.5713 - val_recall_m: 0.8700 - val_f1: 0.3193
Epoch 4/100
11934/11934 [==============================] - 2s 185us/step - loss: 0.7646 - acc: 0.6732 - recall_m: 0.7210 - f1: 0.6858 - val_loss: 0.6226 - val_acc: 0.7999 - val_recall_m: 0.6216 - val_f1: 0.4132
Epoch 5/100
11934/11934 [==========================

11934/11934 [==============================] - 2s 205us/step - loss: 0.5421 - acc: 0.7831 - recall_m: 0.7987 - f1: 0.7863 - val_loss: 0.5417 - val_acc: 0.8123 - val_recall_m: 0.6839 - val_f1: 0.4529
Epoch 36/100
11934/11934 [==============================] - 2s 205us/step - loss: 0.5430 - acc: 0.7770 - recall_m: 0.8018 - f1: 0.7819 - val_loss: 0.5387 - val_acc: 0.8135 - val_recall_m: 0.6765 - val_f1: 0.4521

Epoch 00036: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 37/100
11934/11934 [==============================] - 3s 213us/step - loss: 0.5446 - acc: 0.7779 - recall_m: 0.7897 - f1: 0.7802 - val_loss: 0.5360 - val_acc: 0.8129 - val_recall_m: 0.6729 - val_f1: 0.4499
Epoch 38/100
11934/11934 [==============================] - 3s 212us/step - loss: 0.5477 - acc: 0.7765 - recall_m: 0.7899 - f1: 0.7792 - val_loss: 0.5395 - val_acc: 0.8135 - val_recall_m: 0.6839 - val_f1: 0.4545
Epoch 39/100
11934/11934 [==============================] - 2s 204us/step - loss: 0.

,model,threshold,accuracy,precision,recall,f1,auc_roc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.911475,0.756757,0.383562,0.509091,0.905028,0.602102,"[[528, 9], [45, 28]]",RF0.joblib
1,RF_modT,0.235000,0.842623,0.424837,0.890411,0.575221,0.905028,0.602102,"[[449, 88], [8, 65]]",RF_modT0.joblib
2,RF_modT,0.370000,0.903279,0.597222,0.589041,0.593103,0.904939,0.599707,"[[508, 29], [30, 43]]",RF_modT1.joblib
3,DNN,0.500000,0.806557,0.352941,0.739726,0.477876,0.864200,0.521595,"[[438, 99], [19, 54]]",DNN0.h5
4,DNN_modT,0.714105,0.873770,0.479167,0.630137,0.544379,0.864200,0.521595,"[[487, 50], [27, 46]]",DNN_modT0.h5


Train on 14834 samples, validate on 1931 samples
Epoch 1/100
14834/14834 [==============================] - 3s 206us/step - loss: 1.2072 - acc: 0.4594 - recall_m: 0.2408 - f1: 0.3074 - val_loss: 0.7608 - val_acc: 0.7147 - val_recall_m: 0.1878 - val_f1: 0.0481
Epoch 2/100
14834/14834 [==============================] - 3s 205us/step - loss: 1.1759 - acc: 0.4615 - recall_m: 0.2673 - f1: 0.3311 - val_loss: 0.7936 - val_acc: 0.6339 - val_recall_m: 0.3284 - val_f1: 0.0633
Epoch 3/100
14834/14834 [==============================] - 3s 209us/step - loss: 1.1511 - acc: 0.4533 - recall_m: 0.2772 - f1: 0.3359 - val_loss: 0.8208 - val_acc: 0.5717 - val_recall_m: 0.3552 - val_f1: 0.0591
Epoch 4/100
14834/14834 [==============================] - 3s 206us/step - loss: 1.1259 - acc: 0.4560 - recall_m: 0.2993 - f1: 0.3542 - val_loss: 0.8426 - val_acc: 0.5339 - val_recall_m: 0.3885 - val_f1: 0.0606

Epoch 00004: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.
Epoch 5/100
14834/14834 [

,model,threshold,accuracy,precision,recall,f1,auc_roc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.981229,0.666667,0.166667,0.266667,0.687573,0.274786,"[[573, 1], [10, 2]]",RF0.joblib
1,RF_modT,0.140000,0.965870,0.277778,0.416667,0.333333,0.687573,0.274786,"[[561, 13], [7, 5]]",RF_modT0.joblib
2,DNN,0.500000,0.733788,0.044304,0.583333,0.082353,0.687064,0.062362,"[[423, 151], [5, 7]]",DNN0.h5
3,DNN_modT,0.862187,0.914676,0.104167,0.416667,0.166667,0.687064,0.062362,"[[531, 43], [7, 5]]",DNN_modT0.h5


...

,model,threshold,accuracy,precision,recall,f1,auc_roc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.965574,0.600000,0.136364,0.222222,0.752512,0.245287,"[[586, 2], [19, 3]]",RF0.joblib
1,RF_modT,0.260000,0.945902,0.322581,0.454545,0.377358,0.752512,0.245287,"[[567, 21], [12, 10]]",RF_modT0.joblib
2,DNN,0.500000,0.736066,0.073620,0.545455,0.129730,0.669179,0.096227,"[[437, 151], [10, 12]]",DNN0.h5
3,DNN_modT,0.940785,0.963934,0.500000,0.090909,0.153846,0.669179,0.096227,"[[586, 2], [20, 2]]",DNN_modT0.h5


Train on 10204 samples, validate on 1512 samples
Epoch 1/100
10204/10204 [==============================] - 2s 209us/step - loss: 0.8324 - acc: 0.6445 - recall_m: 0.5566 - f1: 0.6097 - val_loss: 0.6541 - val_acc: 0.7560 - val_recall_m: 0.4507 - val_f1: 0.3642
Epoch 2/100
10204/10204 [==============================] - 2s 211us/step - loss: 0.8260 - acc: 0.6475 - recall_m: 0.5522 - f1: 0.6097 - val_loss: 0.6541 - val_acc: 0.7560 - val_recall_m: 0.4507 - val_f1: 0.3642
Epoch 3/100
10204/10204 [==============================] - 2s 210us/step - loss: 0.8325 - acc: 0.6441 - recall_m: 0.5489 - f1: 0.6063 - val_loss: 0.6541 - val_acc: 0.7560 - val_recall_m: 0.4507 - val_f1: 0.3642
Epoch 4/100
10204/10204 [==============================] - 2s 206us/step - loss: 0.8347 - acc: 0.6410 - recall_m: 0.5511 - f1: 0.6048 - val_loss: 0.6541 - val_acc: 0.7560 - val_recall_m: 0.4507 - val_f1: 0.3642

Epoch 00004: ReduceLROnPlateau reducing learning rate to 3.469447116743502e-21.
Epoch 5/100
10204/10204 [=

,model,threshold,accuracy,precision,recall,f1,auc_roc,avg_precision,confusion_matrix,model_filename
0,RF,0.50000,0.909761,0.677419,0.35,0.461538,0.923982,0.544850,"[[473, 10], [39, 21]]",RF0.joblib
1,RF_modT,0.28000,0.883978,0.486486,0.90,0.631579,0.923982,0.544850,"[[426, 57], [6, 54]]",RF_modT0.joblib
2,DNN,0.50000,0.830571,0.368852,0.75,0.494505,0.847326,0.402595,"[[406, 77], [15, 45]]",DNN0.h5
3,DNN_modT,0.52091,0.836096,0.378151,0.75,0.502793,0.847326,0.402595,"[[409, 74], [15, 45]]",DNN_modT0.h5


Train on 13340 samples, validate on 1781 samples
Epoch 1/100
13340/13340 [==============================] - 3s 261us/step - loss: 0.9043 - acc: 0.6159 - recall_m: 0.5385 - f1: 0.5835 - val_loss: 0.6664 - val_acc: 0.7563 - val_recall_m: 0.4451 - val_f1: 0.1855
Epoch 2/100
13340/13340 [==============================] - 3s 230us/step - loss: 0.9082 - acc: 0.6166 - recall_m: 0.5439 - f1: 0.5862 - val_loss: 0.6664 - val_acc: 0.7563 - val_recall_m: 0.4451 - val_f1: 0.1855
Epoch 3/100
13340/13340 [==============================] - 3s 233us/step - loss: 0.9096 - acc: 0.6205 - recall_m: 0.5515 - f1: 0.5920 - val_loss: 0.6664 - val_acc: 0.7563 - val_recall_m: 0.4451 - val_f1: 0.1855
Epoch 4/100
13340/13340 [==============================] - 3s 221us/step - loss: 0.9079 - acc: 0.6180 - recall_m: 0.5394 - f1: 0.5849 - val_loss: 0.6664 - val_acc: 0.7563 - val_recall_m: 0.4451 - val_f1: 0.1855

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0842022239823443e-22.
Epoch 5/100
13340/13340 [

,model,threshold,accuracy,precision,recall,f1,auc_roc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.935065,0.666667,0.048780,0.090909,0.787402,0.211803,"[[574, 1], [39, 2]]",RF0.joblib
1,RF_modT,0.260000,0.899351,0.266667,0.292683,0.279070,0.787402,0.211803,"[[542, 33], [29, 12]]",RF_modT0.joblib
2,DNN,0.500000,0.745130,0.137500,0.536585,0.218905,0.727762,0.135206,"[[437, 138], [19, 22]]",DNN0.h5
3,DNN_modT,0.493382,0.709416,0.144330,0.682927,0.238298,0.727762,0.135206,"[[409, 166], [13, 28]]",DNN_modT0.h5


# Performance Metrics Saving Development

In [ ]:
p_te = DNN.predict(x_te[rows_te])[:,0]
y_testing=y_te[target][rows_te]
auc_te = roc_auc_score(y_testing, p_te)
print("%15s: %3.5f" % (target, auc_te))

y_hat_testing=DNN.predict_classes(x_te[rows_te])
average_precision=average_precision_score(y_testing,p_te)
mv=evaluate_model_predictions(target,'DNN',0.5,y_testing,y_hat_testing,\
                              auc_te,average_precision)
#     check_and_save(target,mv,DNN,True)
display(mv)
precision, recall, thresholds = precision_recall_curve(y_testing,p_te)
# f1 = 2*precision*recall/(precision+recall)
p,r,t = zip(*[i for i in zip(precision,recall,np.append(thresholds,1)) if i[0:2]!=(0,0)])
p,r,t = np.array(p),np.array(r),np.array(t)
f1 = 2*p*r/(p+r)
m_idx = np.argmax(f1)
m_thresh = thresholds[m_idx]
y_hat_testing_adj=adjusted_classes(p_te,m_thresh)
mv=evaluate_model_predictions(target,'DNN_modT',m_thresh,y_testing,y_hat_testing_adj,\
                              auc_te,average_precision)
#     check_and_save(target,mv,DNN,True)
# display(get_model_perfs(target))
display(mv)

# Troubleshooting Precision-Recall Error

In [ ]:
f1 = 2*precision*recall/(precision+recall)

In [ ]:
a, b = zip(*zip(precision,recall))
type(a)

In [ ]:
p,r,t = zip(*[i for i in zip(precision,recall,np.append(thresholds,1)) if i[0:2]!=(0,0)])
p,r,t = np.array(p),np.array(r),np.array(t)
f1 = 2*p*r/(p+r)

# Testing DNN Model Saving.

In [ ]:
save_model(target,'DNN_test',DNN,True)

In [ ]:
read_model(target,'DNN_test0.h5')

In [ ]:
keras.__version__